In [ ]:
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import scipy.io as scio
import pandas as pd


In [ ]:
dp = '.\\dataset'
names = os.listdir(dp)
print(names)

In [ ]:
names_dict = {'B':[],'IR':[],'OR':[],'normal':[]}
for name in names:
    if 'B' in name: names_dict['B'].append(name)
    if 'IR' in name: names_dict['IR'].append(name)
    if 'OR' in name: names_dict['OR'].append(name)
    if 'normal' in name: names_dict['normal'].append(name)

for c in names_dict.keys():
    print(c,len(names_dict[c]),sep='\t')
    
names_dict[c] = names_dict[c] + ['None']*8
df = pd.DataFrame(names_dict,)
df

# 一个文件名由3个内容组成：“故障类型”+“故障程度”+“第几次实验”

故障严重(qdata)：B/IR/OR的021 + normal=（3+1）组，每组有0/1/2/3次实验

故障轻微(zdata)：B/IR/OR的007 + normal=（3+1）组，每组有0/1/2/3次实验

'B','IR','OR','normal' = 1/2/3/4

In [ ]:
qdata = []
zdata = []

for c in ['B','IR','OR','normal']:
    for item in names_dict[c]:
        if '007' in item:
            qdata.append(item)
            
        if '021' in item:
            zdata.append(item)
            
        if 'normal' in item:
            qdata.append(item)
            zdata.append(item)
            
for i in range(16):
    print(qdata[i],zdata[i],sep='\t')    

# mat数据解析

In [ ]:
print(os.path.join(dp, names[0]))
mat = scio.loadmat(os.path.join(dp, names[0]))
print(list(mat.keys()))

In [ ]:
def find_target_key(target,mat):
    for key in  list(mat.keys()):
        if target in key: break
    return key

find_target_key(target='FE',mat=mat)

In [ ]:
for name in names[::4]:
    mat = scio.loadmat(os.path.join(dp, name))
    DE_key = find_target_key(target='DE',mat=mat)
    FE_key = find_target_key(target='FE',mat=mat)
    
    DE = mat[DE_key]
    FE = mat[FE_key]    
    
    plt.figure(figsize=(8,2))
    plt.plot(DE,label='DE')
#     plt.plot(FE,label='FE')
#     plt.legend()
    plt.ylabel(name[:1])
    plt.savefig(name[:-3],dpi=300)
    plt.show()

# 采样率=12kHz，4000/12000=1/3=0.3秒的数据

In [ ]:
def scale(x):
    mean = np.mean(x)
    std = np.std(x)
    return (x - mean)/std

# def scale(x):
#     mi = np.min(x)
#     ma = np.max(x)
#     return (x - mi)/(ma-mi)*2-1

In [ ]:
qdata_DE = []       # 存储轻(q)故障数据的driven end，DE 数据
qlabel = []     # 存储重(q)故障数据的label
zdata_DE = []       # 存储重(z)故障数据的driven end，DE 数据
zlabel = []     # 存储重(q)故障数据的label
label_2_num = {'B':1,'I':2,'O':3,'n':0}

for name in qdata:
    mat = scio.loadmat(os.path.join(dp, name))
    DE_key = find_target_key(target='DE',mat=mat)
    DE = mat[DE_key]
    DE = scale(DE)
    DE_12 = DE[:120000].reshape(30,-1)
    qdata_DE.append(DE_12) 
    c = label_2_num[name[0]]
    qlabel = qlabel + 30*[c]
    print(name,DE_key,DE.shape,DE_12.shape,sep='\t')
    
qdata_DE = np.concatenate(qdata_DE)
print(qdata_DE.shape,len(qlabel),qlabel[::30])
    
for name in zdata:
    mat = scio.loadmat(os.path.join(dp, name))
    DE_key = find_target_key(target='DE',mat=mat)
    DE = mat[DE_key]
    DE = scale(DE)
    DE_12 = DE[:120000].reshape(30,-1)
    zdata_DE.append(DE_12) 
    c = label_2_num[name[0]]
    zlabel = zlabel + 30*[c]
    print(name,DE_key,DE.shape,DE_12.shape,sep='\t')
    
zdata_DE = np.concatenate(zdata_DE)
print(zdata_DE.shape,len(zlabel),zlabel[::30])

In [ ]:
for i in [0,120,240,360]:
    plt.figure(figsize=(8,1))
    plt.plot(zdata_DE[i])
    plt.ylabel(zlabel[i])
    plt.show()

In [ ]:
qds = tf.data.Dataset.from_tensor_slices((qdata_DE,qlabel)).shuffle(1000)
zds = tf.data.Dataset.from_tensor_slices((zdata_DE,zlabel)).shuffle(1000)
ds = zds
tra_ds = ds.take(360).shuffle(1000).repeat().batch(12) # 取前 360/480 做为训练
tes_ds = ds.skip(360).shuffle(1000).repeat().batch(12) # 取后 （480-360）/480 做为测试

for i,(x,y) in enumerate(tra_ds):
    print(i,x.shape,y.shape,x.dtype,y,sep='\t')
    break
        
x = x.numpy()
y = y.numpy()
print(x.shape,np.mean(y))

for i in range(4):
    plt.figure(figsize=(8,1))
    plt.plot(x[i])
    plt.ylabel(y[i])
    plt.show()
print()

# 构建模型

In [ ]:
dim = qdata_DE.shape[-1]
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(dim,)),

#-------------------------------------卷积网络---------------------------------
    tf.keras.layers.Reshape((dim, 1)),
    tf.keras.layers.Conv1D(4,5,2,activation='relu',padding='same'),
    tf.keras.layers.MaxPool1D(),    
    tf.keras.layers.Conv1D(8,3,2,activation='relu',padding='same'),
    tf.keras.layers.Conv1D(16,3,2,activation='relu',padding='same'),
    tf.keras.layers.Flatten(),
#-------------------------------------卷积网络---------------------------------

#-------------------------------------全连接网络---------------------------------
    tf.keras.layers.Dense(64, activation='relu'),
#-------------------------------------全连接网络---------------------------------
    
    tf.keras.layers.Dropout(0.2),    
    tf.keras.layers.Dense(4, activation='softmax'),
])

model.summary()
tf.keras.utils.plot_model(model,show_shapes=True,show_layer_names=True,dpi=300,)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              # tf.keras.losses.SparseCategoricalCrossentropy(), 'sparse_categorical_crossentropy'
              metrics=['acc'])

h = model.fit(tra_ds,       epochs=20,        steps_per_epoch = 10 ,
                validation_data=tes_ds,        validation_steps=4)

In [ ]:
font = {'family' : 'Times New Roman','weight' : 'normal','size'   : 30,}

figure, ax = plt.subplots(figsize=[10,7])
ax.plot(h.history['loss'], linewidth=2)
ax.plot(h.history['val_loss'], linewidth=2)

ax.legend(['train loss','valid loss'],fontsize=15,loc ='center right')
ax.xaxis.grid(True, linestyle='--', which='major', color='grey', alpha=.2)
ax.yaxis.grid(True, linestyle='--', which='major', color='grey', alpha=.5)

ax.tick_params(labelsize=20)  # 坐标刻度上的字的大小
ax.set_xlabel(xlabel='steps',     fontdict=font)
ax.set_ylabel(ylabel='loss',  fontdict=font)
# ax.set_ylim(0,1)
plt.savefig('loss_steps.png',dpi=300)
plt.show()

figure, ax = plt.subplots(figsize=[10,7])
ax.plot(h.history['acc'], linewidth=2)
ax.plot(h.history['val_acc'], linewidth=2)

ax.legend(['train accuracy','valid accuracy'],fontsize=15,loc ='center right')
ax.xaxis.grid(True, linestyle='--', which='major', color='grey', alpha=.2)
ax.yaxis.grid(True, linestyle='--', which='major', color='grey', alpha=.5)

ax.tick_params(labelsize=20)  # 坐标刻度上的字的大小
ax.set_xlabel(xlabel='steps',     fontdict=font)
ax.set_ylabel(ylabel='accuracy',  fontdict=font)
# ax.set_ylim(0,1)
plt.savefig('acc_steps.png',dpi=300)
plt.show()

In [ ]:
model.evaluate(tra_ds,steps=360//12*5)
model.evaluate(tes_ds,steps=120//12*5)